# Homework 4

In this homework we explore how we can leverage CUDA streams and asynchronous operations to maximize the throughput of matrix-vector multiplication. We build on the matrix-vector CUDA kernel we wrote last week namely the `ShuffleSingleWarpMultiplier` which assigns a single warp to each row and uses `__shfl_down_sync` to reduce the distributed multiplication.

We explore CUDA streams by varying the number of created streams and assigning an approximately equal number of rows to each stream.

All of the relevant code can be found in `homework_4/src/benchmark.cu` of the linked repository.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

benchmark = pd.read_csv("../data/data.csv")